In [ ]:
%%writefile optimized_matmul_triton_day44.py
import torch
import triton
import triton.language as tl

@triton.autotune(
    configs=[
        triton.Config({'BLOCK_SIZE': 16}, num_warps=4),
        triton.Config({'BLOCK_SIZE': 32}, num_warps=8),
    ],
    key=['M', 'N', 'K']
)
@triton.jit
def optimized_matmul_kernel(
    A, B, C, 
    M, N, K, 
    stride_am, stride_ak, 
    stride_bk, stride_bn, 
    stride_cm, stride_cn, 
    BLOCK_SIZE: tl.constexpr
):
    """Optimized matrix multiplication kernel using Triton.
    This kernel is autotuned over BLOCK_SIZE and warp count.
    """
    pid = tl.program_id(0)
    num_cols = N // BLOCK_SIZE
    row = pid // num_cols
    col = pid % num_cols

    a_ptr = A + row * BLOCK_SIZE * stride_am + tl.arange(0, BLOCK_SIZE)[:, None] * stride_ak
    b_ptr = B + tl.arange(0, BLOCK_SIZE)[None, :] * stride_bn + col * BLOCK_SIZE * stride_bk

    acc = tl.zeros((BLOCK_SIZE, BLOCK_SIZE), dtype=tl.float32)

    for k in range(0, K, BLOCK_SIZE):
        a = tl.load(a_ptr + k * stride_ak, mask=(row * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)[:, None]) < M, other=0.0)
        b = tl.load(b_ptr + k * stride_bk, mask=(tl.arange(0, BLOCK_SIZE)[None, :] + k) < K, other=0.0)
        acc += tl.dot(a, b)

    c_ptr = C + row * BLOCK_SIZE * stride_cm + col * BLOCK_SIZE * stride_cn
    mask = (row * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)[:, None]) < M & (col * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)[None, :]) < N
    tl.store(c_ptr, acc, mask=mask)

if __name__ == '__main__':
    # Matrix dimensions (reduce if you want a quick run)
    M, N, K = 1024, 1024, 1024
    BLOCK_SIZE = 16  # autotuner will consider alternatives

    A = torch.randn((M, K), device="cuda", dtype=torch.float32)
    B = torch.randn((K, N), device="cuda", dtype=torch.float32)
    C = torch.empty((M, N), device="cuda", dtype=torch.float32)

    grid = (M // BLOCK_SIZE) * (N // BLOCK_SIZE)

    optimized_matmul_kernel[grid](
        A, B, C, 
        M, N, K, 
        A.stride(0), A.stride(1), 
        B.stride(0), B.stride(1), 
        C.stride(0), C.stride(1), 
        BLOCK_SIZE
    )

    C_ref = torch.matmul(A, B)
    print("Allclose:", torch.allclose(C, C_ref, atol=1e-3))

In [ ]:
# To run the example locally (may require a lot of GPU memory):
# python optimized_matmul_triton_day44.py

# For a quick test, change M,N,K to 128 or 256 in the script above.

## Output:
```
Allclose: True  # (expected)
```
Note: Running the example requires a CUDA-capable GPU, Triton installed, and sufficient memory for the chosen matrix sizes.